In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.5
%matplotlib inline

In [18]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos.drop(["mapacalle", "ciudad"], axis=1, inplace=True)
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [19]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [20]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [21]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [22]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [23]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,idpais,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,...,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,multinacional
0,18,BolNL,0.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

53

In [25]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

126

In [26]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)


postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,ofrece_beneficios,obra_social,multinacional,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
4254905,1112437283,a6E3qq,0,MASC,45.0,-,-,-,-,Graduado,...,False,False,True,2.0,0.0,NaN,0.0,0.0,0.0,False
3399492,1112412665,lDwjLXG,0,MASC,33.0,-,-,-,-,Graduado,...,True,False,False,34.0,0.0,NaN,0.0,22.0,22.0,False
58192,1112412660,4rWVN8R,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,True,104.0,0.0,NaN,1.0,83.0,83.0,False
2168991,1112403061,E6ebe8,1,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,39.0,0.0,NaN,8.0,31.0,31.0,False
2665459,1112268008,vVj8kGG,1,MASC,21.0,-,-,-,-,Graduado,...,False,False,False,1.0,0.0,NaN,0.0,0.0,0.0,False
3474509,1112380097,ZKRAwY,1,MASC,35.0,-,-,-,-,Graduado,...,False,False,False,1.0,0.0,NaN,0.0,1.0,1.0,False
4875721,1112228564,1QrX1mQ,0,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,1.0,0.0,NaN,0.0,1.0,1.0,False
2684192,1112447066,A39bQPL,1,MASC,22.0,-,-,-,-,-,...,True,False,False,6.0,0.0,NaN,0.0,3.0,3.0,False
4407862,1112479578,owjQBEa,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,NaN,0.0,NaN,0.0,0.0,0.0,False
545469,1112297437,GNJW81E,1,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,38.0,1.0,0.026316,14.0,34.0,34.0,True


In [27]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [28]:
postulaciones.head()



,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,ofrece_beneficios,obra_social,multinacional,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
0,1112489014,65x3rR,0,FEM,30.0,-,-,-,-,-,...,False,False,False,1.0,0.0,NaN,0.0,0.0,0.0,False
1,1112489014,2zw4z3v,0,FEM,38.0,-,-,-,En Curso,Graduado,...,False,False,False,13.0,0.0,NaN,1.0,10.0,10.0,False
2,1112489014,LNK8NDw,0,NO_DECLARA,23.0,-,-,-,-,-,...,False,False,False,7.0,0.0,NaN,0.0,5.0,5.0,False
3,1112489014,8MPK0VL,0,FEM,25.0,-,-,-,-,-,...,False,False,False,3.0,0.0,NaN,0.0,3.0,3.0,False
4,1112489014,BmZGBYL,0,FEM,25.0,-,-,-,-,-,...,False,False,False,NaN,0.0,NaN,0.0,0.0,0.0,False


In [29]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)


In [30]:
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
2173188,1112433534,BmDvKXL,0,MASC,25.0,-,-,-,-,Graduado,...,NaN,0.0,NaN,0.0,0.0,0.0,False,1,12,3
4162386,1112362954,JBdrk2p,0,FEM,36.0,-,-,-,-,En Curso,...,1.0,0.0,NaN,0.0,1.0,1.0,False,0,3,0
5442675,1112493522,lDwlxVG,0,FEM,33.0,-,-,En Curso,-,Graduado,...,1.0,0.0,NaN,0.0,1.0,1.0,False,0,7,5
1145194,1112452360,jkjKDKY,1,MASC,26.0,-,-,-,-,Abandonado,...,146.0,1.0,0.006849,10.0,134.0,134.0,True,4,53,44
2549414,1112265650,Rz9jzOO,1,FEM,23.0,-,-,-,-,Graduado,...,1.0,0.0,NaN,0.0,1.0,1.0,False,25,80,66
6368826,1112495510,YV3d31,0,FEM,43.0,-,-,-,-,-,...,1.0,0.0,NaN,0.0,1.0,1.0,False,0,8,2
5214275,1112228625,2zNdMQx,0,FEM,38.0,-,-,-,-,Graduado,...,2.0,0.0,NaN,0.0,1.0,1.0,False,0,5,4
1664115,1112406572,owa6Pmr,1,FEM,27.0,-,-,-,-,-,...,65.0,0.0,NaN,4.0,40.0,40.0,False,4,24,14
798282,1112214523,vV3z3XX,1,MASC,33.0,-,-,-,-,Graduado,...,136.0,1.0,0.007353,64.0,125.0,125.0,True,52,78,55
5051960,1112268218,1BROWb,0,MASC,40.0,-,-,-,-,-,...,NaN,0.0,NaN,0.0,0.0,0.0,False,0,5,3


In [31]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')


x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5754526 Test:  639392


98

In [32]:
model = LGBMClassifier(learning_rate=0.05,objective='binary',num_leaves=12000,max_depth=18,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[1]	valid_0's binary_logloss: 0.661244
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.631444
[3]	valid_0's binary_logloss: 0.603425
[4]	valid_0's binary_logloss: 0.577614
[5]	valid_0's binary_logloss: 0.556091
[6]	valid_0's binary_logloss: 0.534252
[7]	valid_0's binary_logloss: 0.515595
[8]	valid_0's binary_logloss: 0.49842
[9]	valid_0's binary_logloss: 0.481697
[10]	valid_0's binary_logloss: 0.465468
[11]	valid_0's binary_logloss: 0.450445
[12]	valid_0's binary_logloss: 0.436493
[13]	valid_0's binary_logloss: 0.423586
[14]	valid_0's binary_logloss: 0.412146
[15]	valid_0's binary_logloss: 0.40383
[16]	valid_0's binary_logloss: 0.392987
[17]	valid_0's binary_logloss: 0.384269
[18]	valid_0's binary_logloss: 0.374607
[19]	valid_0's binary_logloss: 0.36607
[20]	valid_0's binary_logloss: 0.35762
[21]	valid_0's binary_logloss: 0.349606
[22]	valid_0's binary_logloss: 0.34209
[23]	valid_0's binary_logloss: 0.335142
[24]	valid_0's binary_logloss: 0.

In [37]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9831331129001821


In [38]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/MegaLGBM1.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/MegaLGBM1.pkl']

In [ ]:
vistas3.head()

In [44]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
85860,35316,1112410217,50R2w,FEM,42.0,-,-,-,-,Graduado,...,92.0,0.0,0.000000,False,0.0,15.0,12.0,1.0,80.0,80.0
81536,29273,1112370444,6Aa8Z3,FEM,34.0,-,-,Graduado,-,-,...,12.0,0.0,0.000000,False,0.0,9.0,10.0,0.0,8.0,8.0
64464,12155,1111984880,eVmKWB,MASC,34.0,-,-,-,-,-,...,13.0,0.0,0.000000,False,1.0,25.0,12.0,1.0,11.0,11.0
40077,57876,1112451947,KBA1eNd,MASC,26.0,-,-,-,-,Graduado,...,41.0,0.0,0.000000,False,4.0,46.0,32.0,3.0,41.0,41.0
55835,47112,1112431087,qekjzpL,FEM,25.0,-,-,En Curso,-,Graduado,...,131.0,1.0,0.007634,True,33.0,75.0,17.0,40.0,94.0,94.0
2925,74778,1112463897,wVkrVpQ,FEM,23.0,-,-,-,-,Abandonado,...,117.0,1.0,0.008547,True,4.0,13.0,10.0,37.0,63.0,63.0
2285,173,1731810,8BrRZL,MASC,34.0,-,En Curso,-,-,Graduado,...,3.0,0.0,0.000000,False,0.0,7.0,1.0,0.0,3.0,3.0
99894,99879,1112514169,50RedM,FEM,35.0,-,-,-,-,-,...,8.0,0.0,0.000000,False,0.0,9.0,8.0,0.0,8.0,8.0
38833,3761,1111413600,KBrqzKq,FEM,26.0,-,-,-,-,Graduado,...,24.0,1.0,0.041667,True,6.0,6.0,3.0,9.0,8.0,8.0
2856,74763,1112463897,awWwV9,FEM,36.0,-,-,-,-,-,...,73.0,1.0,0.013699,True,5.0,9.0,8.0,21.0,39.0,39.0


In [45]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,0.0,0.0,False,0.0,3.0,0.0,3.0,20.0,20.0,0.010660
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,0.0,0.0,False,3.0,25.0,0.0,0.0,3.0,3.0,0.230727
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,0.0,0.0,False,1.0,10.0,0.0,1.0,6.0,6.0,0.057789
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,0.0,0.0,False,0.0,9.0,0.0,0.0,0.0,0.0,0.002264
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,0.0,0.0,False,0.0,7.0,2.0,0.0,1.0,1.0,0.003077


In [46]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.5002451669620064


,id,sepostulo
0,0,0.010660
1,1,0.230727
2,2,0.057789
3,3,0.002264
4,4,0.003077


In [47]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/mega-lgbm1.csv",index=False)